In [ ]:
!kaggle competitions download -c enerjisa-uretim-hackathon

from zipfile import ZipFile
import os
for i in os.listdir('C:/Users/mehmu/Desktop/enerjisa_datathon/'):
    if '.zip' in i:
        print(i)
        with ZipFile(i, 'r') as zipObj:
            zipObj.extractall()
        os.remove(i)


In [3]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
%matplotlib inline
df=pd.read_csv('features.csv')
df

,Timestamp,Gearbox_T1_High_Speed_Shaft_Temperature,Gearbox_T3_High_Speed_Shaft_Temperature,Gearbox_T1_Intermediate_Speed_Shaft_Temperature,Temperature Gearbox Bearing Hollow Shaft,Tower Acceleration Normal,Gearbox_Oil-2_Temperature,Tower Acceleration Lateral,Temperature Bearing_A,Temperature Trafo-3,...,Blade-1 Actual Value_Angle-A,Blade-2 Set Value_Degree,Pitch Demand Baseline_Degree,Blade-1 Set Value_Degree,Blade-3 Set Value_Degree,Moment Q Direction,Moment Q Filltered,Proxy Sensor_Degree-45,Turbine State,Proxy Sensor_Degree-315
0,2019-01-01 00:00:00,57.000000,59.000000,52.000000,56.158333,47.053776,57.000000,18.890772,35.000000,48.576668,...,0.055473,0.267122,0.000000,0.058617,-0.325738,37.867054,39.281124,5.732657,1.0,5.779913
1,2019-01-01 00:10:00,57.693333,59.485001,52.450001,57.000000,64.969742,57.413334,28.560265,35.233334,48.000000,...,0.055473,0.267122,0.000000,0.058617,-0.325738,-90.169106,-89.018669,5.813177,1.0,5.796757
2,2019-01-01 00:20:00,59.000000,60.756668,53.536667,57.775002,51.149670,58.728333,34.228813,36.000000,48.053333,...,0.055473,0.267122,0.000000,0.058617,-0.325738,-88.556343,-88.422020,5.786413,1.0,5.772958
3,2019-01-01 00:30:00,59.881668,61.563332,54.413334,58.683334,58.740929,59.518333,35.593220,36.000000,48.788334,...,0.055473,0.267122,0.000000,0.058617,-0.325738,-123.755341,-124.858444,5.758913,1.0,5.728393
4,2019-01-01 00:40:00,61.290001,62.586666,55.485001,59.623333,53.264774,60.665001,38.552731,36.000000,49.000000,...,0.053982,0.267612,0.000000,0.058703,-0.325443,-142.533325,-142.053543,5.724591,1.0,5.687730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154257,2021-12-14 23:10:00,65.811668,NaN,59.945000,62.808334,225.038239,65.300003,109.889709,61.000000,97.000000,...,NaN,15.820095,15.199166,15.235223,14.540556,-29.340843,-27.513502,5.746916,1.0,5.756082
154258,2021-12-14 23:20:00,68.586670,NaN,62.084999,65.413330,229.905838,67.871666,106.016670,61.116665,97.000000,...,NaN,16.504293,15.876278,15.917643,15.207320,-31.925669,-30.197918,5.749150,1.0,5.755406
154259,2021-12-14 23:30:00,63.746666,NaN,59.965000,64.051666,223.352631,64.461670,111.690208,61.293335,97.000000,...,NaN,15.331903,14.720088,14.768394,14.064686,-53.071564,-48.306511,5.751807,1.0,5.747936
154260,2021-12-14 23:40:00,66.643333,NaN,60.678333,63.421665,227.704514,66.081665,119.716499,60.786667,97.000000,...,NaN,16.481724,15.887610,15.945046,15.230121,-28.747763,-23.844364,5.747686,1.0,5.757787


In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
from catboost import CatBoostRegressor

IMPUTE = False

In [5]:
def analyze_column(input_series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(input_series):
        return 'numeric'
    else:
        return 'categorical'
    


class LGBMImputer:
    '''
    Regression imputer using LightGBM
    '''
    def __init__(self, cat_features=[], n_iter=15000, verbose=False):
        self.n_iter = n_iter
        self.cat_features = cat_features
        self.verbose = verbose
        self.n_features = None
        self.feature_names = None
        self.feature_with_missing = None
        self.imputers = {}
        self.offsets = {}
        self.objectives = {} 
                                                                            # Özellikler-Parametreler belirleniyor...
        
    def fit_transform(self, X, y=None):
        output_X = X.copy()
        self.n_features = X.shape[1]

        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
        else:
            self.feature_names = [f'f{i}' for i in range(self.n_features)]
            X = pd.DataFrame(X, columns=self.feature_names)
        self.feature_with_missing = [col for col in self.feature_names if X[col].isnull().sum() > 0]

        for icol, col in enumerate(self.feature_with_missing):              #icol sıra numarası, col kolon
            if icol in self.cat_features:
                nuni = X[col].dropna().nunique()
                if nuni == 2:
                    params = {
                        'objective': 'binary'
                    }
                elif nuni > 2:
                    params = {
                        'objective': 'multiclass',
                        'num_class': nuni + 1
                    }
            else: # automatic analyze column
                if analyze_column(X[col]) == 'numeric':
                    params = {
                        'objective': 'regression'
                    }
                else:
                    nuni = X[col].dropna().nunique()
                    if nuni == 2:
                        params = {
                            'objective': 'binary'
                        }
                    elif nuni > 2:
                        params = {
                            'objective': 'multiclass',
                            'num_class': nuni + 1
                        }
                    else:
                        print(f'column {col} has only one unique value.')
                        continue

            params['verbosity'] = -1
            
            null_idx = X[col].isnull()
            x_train = X.loc[~null_idx].drop(col, axis=1)
            x_test = X.loc[null_idx].drop(col, axis=1)
            y_offset = X[col].min()
            y_train = X.loc[~null_idx, col].astype(int) - y_offset
            dtrain = lgb.Dataset(
                data=x_train,
                label=y_train
            )

            early_stopping_rounds = 50
            model = lgb.train(
                params, dtrain, valid_sets=[dtrain], 
                num_boost_round=self.n_iter,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=0,
            )

            y_test = model.predict(x_test)
            if params['objective'] == 'multiclass':
                y_test = np.argmax(y_test, axis=1).astype(float)
            elif params['objective'] == 'binary':
                y_test = (y_test > 0.5).astype(float)
            y_test += y_offset
            output_X.loc[null_idx, col] = y_test
            if params['objective'] in ['multiclass', 'binary']:
                output_X[col] = output_X[col].astype(int)
            self.imputers[col] = model
            self.offsets[col] = y_offset
            self.objectives[col] = params['objective']
            if self.verbose:
                print(f'{col}:\t{self.objectives[col]}...iter{model.best_iteration}/{self.n_iter}')
        
        return output_X    

In [ ]:
if IMPUTE:
    FEATURE_CSV_PATH = "features.csv"
    POWER_CSV_PATH = "power.csv"

    feature_df = pd.read_csv(FEATURE_CSV_PATH)
    power_df = pd.read_csv(POWER_CSV_PATH)
    feature_df = feature_df.merge(power_df, how="left", on="Timestamp")
    
    feature_df = feature_df.replace(99999.0,np.nan)

    imputer = LGBMImputer(verbose=True)
    feature_df.loc[:, feature_df.columns[1:-1]] = imputer.fit_transform(feature_df[feature_df.columns[1:-1]])

    feature_df["month"] = pd.to_datetime(feature_df["Timestamp"]).dt.month
    feature_df["year"] = pd.to_datetime(feature_df["Timestamp"]).dt.year
    feature_df["hour"] = pd.to_datetime(feature_df["Timestamp"]).dt.hour
    feature_df["week"] = pd.to_datetime(feature_df["Timestamp"]).dt.week

    test_mask = pd.to_datetime(feature_df["Timestamp"]) > "2021-08-14 23:50:00"

    train_df = feature_df[~test_mask].copy().reset_index(drop=True)
    test_df = feature_df[test_mask].copy().reset_index(drop=True)
    
else:
    TRAIN_CSV_PATH = "train_imputed.csv"
    TEST_CSV_PATH = "test_imputed.csv"
    SUBM_CSV_PATH = "sample_submission.csv"

    train_df = pd.read_csv(TRAIN_CSV_PATH).set_index("Timestamp")
    test_df = pd.read_csv(TEST_CSV_PATH).set_index("Timestamp")
    subm_df = pd.read_csv(SUBM_CSV_PATH).drop(columns="Power(kW)", axis=1)

In [2]:
label = "Power(kW)"
except_cols = ["year", "month", "week", "hour"]
X_train = train_df.drop(labels = except_cols + [label], axis=1)
y_train = train_df[label]

NameError: name 'train_df' is not defined

We are being evaluated with RMSE in this competition. So how far our predicted values are from the ground truths are crucial. We need to keep that in mind while executing our cross-validations. Let's look at the distribution of our label.

In [ ]:
plt.figure(figsize=(12,6))
_ = sbn.histplot(data=y_train, kde=True)

We can see that most values are clustered at two opposite ends of the distribution. It may be advantageous to maintain this distribution in all our folds.

In [ ]:
def create_cont_folds(df, n_s=8, n_grp=1000):
    
    skf = StratifiedKFold(n_splits=n_s, shuffle=True, random_state=1337)
    grp = pd.cut(df, n_grp, labels=False)
    target = grp
    
    fold_nums = np.zeros(len(df))
    for fold_no, (t, v) in enumerate(skf.split(target, target)):
        fold_nums[v] = fold_no
        
    return fold_nums

In [ ]:
num_of_folds = 8
num_of_groups = 40

foldnums = create_cont_folds(y_train, n_s=num_of_folds, n_grp=num_of_groups)
cv_splits = []

for i in range(num_of_folds):
    test_indices = np.argwhere(foldnums==i).flatten()
    train_indices = list(set(range(len(y_train))) - set(test_indices))
    cv_splits.append((train_indices, test_indices))

In [ ]:
PLOT_FOLD_NUM = 2

fig, axs = plt.subplots(2, figsize=(12,10))
sns.histplot(data=y_train.iloc[cv_splits[PLOT_FOLD_NUM][0]], kde=True, ax=axs[0]).set(title='Train Set Distribution')
sns.histplot(data=y_train.iloc[cv_splits[PLOT_FOLD_NUM][1]], kde=True, ax=axs[1]).set(title='Validation Set Distribution')
plt.show()

In [ ]:
cv_models = []
rms_errs = []

cat_preds = []
lgb_preds = []

cat_final_preds = []
lgb_final_preds = []

lgb_params = {
    "learning_rate": 0.02,
    "num_leaves": 64,
    "colsample_bytree": 0.9,
    "subsample": 0.9,
    "verbosity": -1,
    "n_estimators": 7000,
    "early_stopping_rounds": 50,
    "random_state": 42,
    "objective": "regression",
    "metric": "rmse",
}

for split_train, split_val in tqdm(cv_splits):
    split_train = X_train.index[split_train]
    split_val = X_train.index[split_val]

    model1 = CatBoostRegressor(
        iterations=5000,
        random_state=42,
        early_stopping_rounds=50,
    )

    model2 = lgb.LGBMRegressor(**lgb_params)
 
    train_x, train_y = X_train.loc[split_train], y_train.loc[split_train]
    test_x, test_y = X_train.loc[split_val], y_train.loc[split_val]

    model1.fit(
        train_x,
        train_y,
        eval_set=(test_x, test_y),
        verbose=500,
    )
    model2.fit(
        train_x,
        train_y,
        eval_set=(test_x, test_y),
        verbose=500,
    )

    preds = (model1.predict(test_x) + model2.predict(test_x)) / 2
    rms = mean_squared_error(test_y, preds, squared=False)
    rms_errs.append(rms)

    cat_preds.append(model1.predict(test_x))
    lgb_preds.append(model2.predict(test_x))
    
    cat_final_preds.append(
        model1.predict((test_df.drop(labels=except_cols + [label], axis=1)))
    )
    
    lgb_final_preds.append(
        model2.predict((test_df.drop(labels=except_cols + [label], axis=1)))
    )

    print("RMSE:", rms)

In [ ]:
cat_final_preds = [pd.DataFrame(cat_final_preds).mean(axis=0).values]
lgb_final_preds = [pd.DataFrame(lgb_final_preds).mean(axis=0).values]
two_stage_feats = (
    pd.DataFrame(
        {"cat_pred": np.concatenate(cat_preds), "lgb_pred": np.concatenate(lgb_preds)},
        index=pd.concat(
            [
                pd.Series(X_train.loc[X_train.index[test_indices]].index)
                for (_, test_indices) in cv_splits
            ]
        ),
    )
    .reset_index()
    .set_index("Timestamp")
)

two_stage_preds = (
    pd.DataFrame(
        {
            "cat_pred": np.concatenate(cat_final_preds),
            "lgb_pred": np.concatenate(lgb_final_preds),
        },
        index=(test_df.drop(labels=except_cols + [label], axis=1)).index,
    )
    .reset_index()
    .set_index("Timestamp")
)

In [ ]:
rms_errs = []

final_preds = []
cat_models = []
lgb_models = []

lgb_params = {
    "learning_rate": 0.02,
    "num_leaves": 64,
    "colsample_bytree": 0.9,
    "subsample": 0.9,
    "verbosity": -1,
    "n_estimators": 7000,
    "early_stopping_rounds": 50,
    "random_state": 42,
    "objective": "regression",
    "metric": "rmse",
}


for split_train, split_val in tqdm(cv_splits):
    split_train = X_train.index[split_train]
    split_val = X_train.index[split_val]

    model1 = CatBoostRegressor(
        iterations=5000,
        random_state=42,
        early_stopping_rounds=50,
    )

    model2 = lgb.LGBMRegressor(**lgb_params)

    train_x, train_y = (
        pd.concat([X_train.loc[split_train], two_stage_feats.loc[split_train]], axis=1),
        y_train.loc[split_train],
    )
    test_x, test_y = (
        pd.concat([X_train.loc[split_val], two_stage_feats.loc[split_val]], axis=1),
        y_train.loc[split_val],
    )

    model1.fit(
        train_x,
        train_y,
        eval_set=(test_x, test_y),
        verbose=500,
    )
    
    model2.fit(
        train_x,
        train_y,
        eval_set=(test_x, test_y),
        verbose=500,
    )

    preds = (model1.predict(test_x) + model2.predict(test_x)) / 2
    rms = mean_squared_error(test_y, preds, squared=False)
    rms_errs.append(rms)

    final_preds.append(
        (
            model1.predict(
                pd.concat(
                    [
                        (test_df.drop(labels=except_cols + [label], axis=1)),
                        two_stage_preds,
                    ],
                    axis=1,
                )
            )
            + model2.predict(
                pd.concat(
                    [
                        (test_df.drop(labels=except_cols + [label], axis=1)),
                        two_stage_preds,
                    ],
                    axis=1,
                )
            )
        )
        / 2
    )

    print("RMSE:", rms)

In [ ]:
print("Current Submission CV:", rms_errs)
print("Current Submission CV Mean:", np.array(rms_errs).mean())
print("Current Submission CV Std:", np.array(rms_errs).std())

In [ ]:
pd.DataFrame(final_preds, columns=test_df.index).mean(axis=0).to_frame(
    "Power(kW)"
).reset_index().to_csv("submission.csv", index=False)